In [10]:
import ctypes
import sys
import win32process
import win32gui
import win32api

In [11]:
# Definindo algumas constantes e variáveis necessárias
window_name = "Perfect World - Rate Media: Jornada ao Oeste"
moblist_filename = "moblist.dat"
target_list = [0] * 255
target_num = 0

In [12]:
# Definindo os offsets (esses valores precisam ser definidos corretamente)
addr_character_struc = 0
addr_character_struc_actual = 0

addr_base_static = 0x009B4594 #0x0092764C
offset_from_base_for_coords = 0x14
offset_hp = 1104
offset_mp = 1108
offset_maxhp = 1144
offset_maxmp = 1148
offset_target = 2800 #2584
offset_exp = 1104
offset_maxexp = 1104

In [13]:
# Estruturas de dados necessárias
class ProcessMemory:
    def __init__(self, pid):
        self.pid = pid
        self.handle = win32api.OpenProcess(0x1F0FFF, False, self.pid)

    def read_memory(self, address, size):
        # Corrigindo a chamada de ReadProcessMemory
        win32process.ReadProcessMemory(self.handle, address, size)

In [14]:
# Encontra a janela do Perfect World
hwnd = win32gui.FindWindow(None, window_name)
if not hwnd:
    print("Unable to find Perfect World window! FindWindow() error.")
    sys.exit(0)

# Obtém o PID do processo
pid = win32process.GetWindowThreadProcessId(hwnd)[1]

In [3]:
import pymem
import psutil


In [3]:
def bytes_to_int(byte_sequence):
    # Converte a sequência de bytes para um número inteiro (32 bits, little-endian)
    return int.from_bytes(byte_sequence, byteorder='little')

In [17]:
process = psutil.Process(pid)
# Obter uso de memória
memory_info = process.memory_info()
print(f"Uso de memória: {memory_info.rss} bytes")


Uso de memória: 1596420096 bytes


In [18]:
process = psutil.Process(pid)
# Obter uso de memória
memory_info = process.memory_info()
print(f"Uso de memória: {memory_info.rss} bytes")


pm = pymem.Pymem(pid)

base_address = 0x6F327EE4

offset = 678288   # Deslocamento fixo conhecido

# Calcular o endereço final
final_address = base_address + offset

try:
    data = pm.read_bytes(final_address, 4)  # Lê 4 bytes a partir do endereço
    print(f"Dados lidos: {data}")
    print(f"Valor como inteiro: {bytes_to_int(data)}")
except pymem.exception.MemoryReadError as e:
    print(f"Erro ao ler memória: {e}")
except pymem.exception.WinAPIError as e:
    print(f"Erro na API do Windows: {e}")


Uso de memória: 1596420096 bytes
Erro ao ler memória: Could not read memory at: 1866258548, length: 4 - GetLastError: 299


In [ ]:
def find_pattern(pm, start_address, pattern, direction='forward', max_size=1024**3, step_size=0x10):
    """
    Função para procurar um padrão específico dentro da memória do processo.
    :param pm: Objeto pymem para o processo
    :param start_address: Endereço inicial da busca
    :param pattern: Padrão a ser buscado
    :param direction: 'forward' para buscar para frente, 'backward' para buscar para trás
    :param max_size: Tamanho máximo da memória a ser vasculhada (default 1GB)
    :param step_size: Tamanho do passo para a busca (default 16 bytes)
    :return: Endereço onde o padrão foi encontrado, ou None se não encontrado
    """
    # Definir o limite superior e inferior dependendo da direção da busca
    if direction == 'forward':
        end_address = start_address + max_size
        step = step_size
    elif direction == 'backward':
        end_address = start_address - max_size
        step = -step_size
    else:
        raise ValueError("A direção deve ser 'forward' ou 'backward'.")
    
    # Iterar pelos endereços de memória, lendo em blocos
    for address in range(start_address, end_address, step):
        try:
            data = pm.read_bytes(address, len(pattern))
            if data == pattern:
                # Calcula o offset em relação ao endereço base
                offset = address - start_address
                print(f"Padrão encontrado no endereço: {hex(address)}")
                print(f"Offset: {offset} bytes")
                return address  # Padrão encontrado, retorna o endereço base
        except pymem.exception.MemoryReadError:
            continue  # Se não conseguir ler, continua a busca
    return None

# Abrir o processo
process = psutil.Process(pid)
pm = pymem.Pymem(pid)

# Padrão que você deseja procurar
pattern = b'A:\x00\x00'  # O padrão que você quer encontrar

# Endereço inicial da busca (6F327EE4)
start_address = 0x140000000

# Buscar o padrão na memória a partir do endereço e vasculhando até 1GB para frente
base_address = find_pattern(pm, start_address, pattern, direction='forward', max_size=1024**3, step_size=0x10)

if base_address:
    print(f"Padrão encontrado no endereço base (para frente): {hex(base_address)}")
else:
    print("Padrão não encontrado dentro do intervalo de busca para frente.")

In [287]:
# Buscar o padrão na memória a partir do endereço e vasculhando até 1GB para trás
base_address = find_pattern(pm, start_address, pattern, direction='backward', max_size=1024**3, step_size=0x10)

if base_address:
    print(f"Padrão encontrado no endereço base (para trás): {hex(base_address)}")
else:
    print("Padrão não encontrado dentro do intervalo de busca para trás.")

Padrão não encontrado dentro do intervalo de busca para trás.


# Rascunyho

In [11]:
# # Encontra a janela do Perfect World
# hwnd = win32gui.FindWindow(None, window_name)
# if not hwnd:
#     print("Unable to find Perfect World window! FindWindow() error.")
#     sys.exit(0)

# # Obtém o PID do processo
# pid = win32process.GetWindowThreadProcessId(hwnd)[1]

In [1]:
import pymem

In [4]:
pm = pymem.Pymem("elementclient_64.exe")
base_address = pm.base_address
print(f"Endereço base: {hex(base_address)}")

Endereço base: 0x140000000


In [5]:
real_address = base_address + 0x00148A08
print(f"Endereço real: {hex(real_address)}")

Endereço real: 0x140148a08


In [6]:
data = pm.read_bytes(0x00148A08, 4)  # Lê 4 bytes a partir do endereço
print(f"Dados lidos: {data}")
print(f"Valor como inteiro: {bytes_to_int(data)}")

Dados lidos: b'\n\x00\x00\x00'
Valor como inteiro: 10


In [306]:
hex(0x520FFEB0 - 0x140000000)

'-0xedf00150'

In [ ]:
import frida
import sys

def on_message(message, data):
    print("[Mensagem do Script]", message)

try:
    # Anexa ao processo "elementclient_64.exe"
    session = frida.attach("elementclient_64.exe")
except Exception as e:
    print("Erro ao anexar no processo:", e)
    sys.exit(1)

# Script JavaScript que será injetado
script_code = """
// Obtém o endereço base do módulo
var moduleBase = Module.findBaseAddress("elementclient_64.exe");
if (moduleBase === null) {
    send("Não foi possível encontrar o endereço base do módulo.");
} else {
    // Calcula o endereço da instrução alvo: elementclient_64.exe + 0x8BACC3
    var targetAddress = moduleBase.add(0x8BACC3);
    send("Instrução alvo: " + targetAddress);
    
    // Armazena o objeto do hook numa variável para que possamos removê-lo posteriormente
    var hook = Interceptor.attach(targetAddress, {
        onEnter: function(args) {
            // Calcula o endereço do HP a partir do valor de RDI + 0x92C
            var hpAddress = this.context.rdi.add(0x92C);
            // Lê um inteiro (32 bits) a partir desse endereço
            var hpValue = Memory.readInt(hpAddress);
            send("RDI: " + this.context.rdi + " | HP Address: " + hpAddress + " | HP Value: " + hpValue);
            // Remove o hook para que não entre em loop
            hook.detach();
        }
    });
}
"""

# Cria e carrega o script
script = session.create_script(script_code)
script.on("message", on_message)
script.load()

print("Script injetado. Aguardando a execução da instrução interceptada...")
sys.stdin.read()


[Mensagem do Script] {'type': 'send', 'payload': 'Instrução alvo: 0x1408bacc3'}
Script injetado. Aguardando a execução da instrução interceptada...
[Mensagem do Script] {'type': 'send', 'payload': 'RDI: 0x51ddbe90 | HP Address: 0x51ddc7bc | HP Value: 15685'}
